# Data Cleaning

In [ ]:
%reset

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import os
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
from math import sqrt
from datetime import date
import holidays
sns.set()
import warnings
warnings.filterwarnings("ignore")

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.sans-serif"] = "Verdana"
mpl.rcParams["lines.markersize"] = 20
# mpl.rc('axes', labelsize=14)
# mpl.rc('xtick', labelsize=12)
# mpl.rc('ytick', labelsize=12)

## Smart Freeways Data 2018 - Links All Days

In [13]:
df = pd.read_excel("data/Smart Freeways Data 2018 - Links All Days.xlsx",
                   sheet_name='NPI Links - Volume', skiprows=2, header=0)
df2 = pd.read_excel("data/Smart Freeways Data 2018 - Links All Days.xlsx",
                   sheet_name='NPI Links - Speed', skiprows=2, header=0)
df3 = pd.read_excel("data/Smart Freeways Data 2018 - Links All Days.xlsx",
                   sheet_name='NPI Links - Occupancy', skiprows=2, header=0)

### Unpivot dataframe

In [78]:
cols1 = df.columns[:3]
cols2 = [d.strftime('%d %b %y') for d in pd.date_range(start='2018-01-01',
                                               end='2018-10-25',
                                               freq='D')]
cols = [*cols1, *cols2]; cols

['Row Labels',
 'NPI Link Length',
 'Hour Formatted',
 '01 Jan 18',
 '02 Jan 18',
 '03 Jan 18',
 '04 Jan 18',
 '05 Jan 18',
 '06 Jan 18',
 '07 Jan 18',
 '08 Jan 18',
 '09 Jan 18',
 '10 Jan 18',
 '11 Jan 18',
 '12 Jan 18',
 '13 Jan 18',
 '14 Jan 18',
 '15 Jan 18',
 '16 Jan 18',
 '17 Jan 18',
 '18 Jan 18',
 '19 Jan 18',
 '20 Jan 18',
 '21 Jan 18',
 '22 Jan 18',
 '23 Jan 18',
 '24 Jan 18',
 '25 Jan 18',
 '26 Jan 18',
 '27 Jan 18',
 '28 Jan 18',
 '29 Jan 18',
 '30 Jan 18',
 '31 Jan 18',
 '01 Feb 18',
 '02 Feb 18',
 '03 Feb 18',
 '04 Feb 18',
 '05 Feb 18',
 '06 Feb 18',
 '07 Feb 18',
 '08 Feb 18',
 '09 Feb 18',
 '10 Feb 18',
 '11 Feb 18',
 '12 Feb 18',
 '13 Feb 18',
 '14 Feb 18',
 '15 Feb 18',
 '16 Feb 18',
 '17 Feb 18',
 '18 Feb 18',
 '19 Feb 18',
 '20 Feb 18',
 '21 Feb 18',
 '22 Feb 18',
 '23 Feb 18',
 '24 Feb 18',
 '25 Feb 18',
 '26 Feb 18',
 '27 Feb 18',
 '28 Feb 18',
 '01 Mar 18',
 '02 Mar 18',
 '03 Mar 18',
 '04 Mar 18',
 '05 Mar 18',
 '06 Mar 18',
 '07 Mar 18',
 '08 Mar 18',
 '09 Mar

In [81]:
def unpivot_data(df):
    # Missing Row Labels and NPI Link Length filled 
    df[['Row Labels', 'NPI Link Length']] = df[['Row Labels', 'NPI Link Length']].fillna(method='ffill')
    
    # Keep rows that have %H:%M time format
    df = df[df['Hour Formatted'].notna()]
    df = df[df['Hour Formatted'].str.len() <= 5]
    
    # Add 25 Mar 18 and/or 26 Mar 18 if one of either of them do not exist
    df = df.reindex(df.columns.union(cols, sort=False), axis=1)
    
    # Change date columns' format from %d %b %y to %Y-%m-%d datetime
    aslist = df.columns.tolist()
    aslist[3:] = pd.to_datetime(df.columns[3:])
    df.columns = aslist
    
    # Change Hour Formatted column to datetime object
    df['Hour Formatted'] = pd.to_datetime(df['Hour Formatted'],
                                              format='%H:%M')
    
    # Melt (unpivot) dataframe
    df = df.melt(id_vars=['Row Labels', 'NPI Link Length', 'Hour Formatted'],
                  var_name='DateTime', value_name='Value')
    # Create DateTime column from date columns and HourFormatted
    df['DateTime'] = (pd.to_datetime(df['DateTime']) +  
                       pd.to_timedelta(df.pop('Hour Formatted')
                                       .dt.strftime('%H:%M:%S')))
    # Sort dataframe by Row Labels and DateTime
    df = df.sort_values(['Row Labels', 'DateTime'], 
                          ascending=[True, True])
    df.reset_index(drop=True, inplace=True)
    
    return df

In [82]:
vol_df = unpivot_data(df)  # 1 Jan 2018 - 1 Nov 2018
spe_df = unpivot_data(df2) # 1 Jan 2018 - 25 Oct 2018
occ_df = unpivot_data(df3) # 1 Jan 2018 - 25 Oct 2018

In [83]:
vol_df.rename({'Row Labels': 'Link',
               'NPI Link Length': 'Length',
               'Value': 'Volume'}, axis=1, inplace=True)
spe_df.rename({'Row Labels': 'Link',
               'NPI Link Length': 'Length',
               'Value': 'Speed'}, axis=1, inplace=True)
occ_df.rename({'Row Labels': 'Link',
               'NPI Link Length': 'Length',
               'Value': 'Occupancy'}, axis=1, inplace=True)

In [84]:
vol_df # 305 days * 1440 mins * 14 Links = 6148800

,Link,Length,DateTime,Volume
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,16.0
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,15.0
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,15.0
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,15.0
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,14.0
...,...,...,...,...
6148795,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-11-01 23:55:00,5.0
6148796,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-11-01 23:56:00,5.0
6148797,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-11-01 23:57:00,5.0
6148798,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-11-01 23:58:00,5.0


In [85]:
spe_df # 298 days * 1440 mins * 14 Links = 6007680

,Link,Length,DateTime,Speed
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,80.000000
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,93.999998
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,93.999998
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,95.000000
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,96.999999
...,...,...,...,...
6007675,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:55:00,96.999997
6007676,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:56:00,96.999997
6007677,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:57:00,94.000001
6007678,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:58:00,94.000001


In [86]:
occ_df # 298 days * 1440 mins * 14 Links = 6007680

,Link,Length,DateTime,Occupancy
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,1.8
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,1.8
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,1.8
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,1.8
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,1.8
...,...,...,...,...
6007675,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:55:00,0.0
6007676,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:56:00,0.0
6007677,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:57:00,0.0
6007678,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:58:00,0.0


### Merge Volume, Speed and Occupancy

In [87]:
# Merge volume and speed
dataset = vol_df.merge(spe_df, on=['Link','Length','DateTime'])

# Merge dataset and occupancy
dataset = dataset.merge(occ_df, on=['Link','Length','DateTime'])

In [88]:
dataset

,Link,Length,DateTime,Volume,Speed,Occupancy
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,16.0,80.000000,1.8
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,15.0,93.999998,1.8
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,15.0,93.999998,1.8
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,15.0,95.000000,1.8
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,14.0,96.999999,1.8
...,...,...,...,...,...,...
6007675,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:55:00,5.0,96.999997,0.0
6007676,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:56:00,5.0,96.999997,0.0
6007677,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:57:00,5.0,94.000001,0.0
6007678,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:58:00,5.0,94.000001,0.0


The new dataframe consists of 298 days $\times$ 1440 mins $\times$ 14 links = 6007680 rows

In [94]:
dataset.Link.unique()

array(['Kwinana Fwy NB between Cranford Av On - H015 Nth Bou & H015 Sth Bound - H548',
       'Kwinana Fwy NB between Farrington Rd On - H015 Nth Bo & H015 Nth Bound - South St Off',
       'Kwinana Fwy NB between H015 Nth Bound - Leach Hwy Off & Leach Hwy West Bound On - H015',
       'Kwinana Fwy NB between H015 Nth Bound - South St Off & South St On - H015 Nth Bound',
       'Kwinana Fwy NB between H015 Sth Bound - H548 & Manning Rd - H547 On Kwinana Fwy Nth Bound',
       'Kwinana Fwy NB between Kwinana Fwy (northbound) Bus Ln From Canning Hwy: H013 On To H015 Northbound',
       'Kwinana Fwy NB between Kwinana Fwy Nth Bnd H015 Nth Bnd - H018 East Bnd & Kwinana Fwy Nth Bnd H018 W',
       'Kwinana Fwy NB between Kwinana Fwy Nth Bnd H018 West Bnd - H015 Nth Bnd & Farrington Rd On - H015 Nt',
       'Kwinana Fwy NB between Kwinana Fwy Nth Bound H503 Off - Mill Pt Rd & Mitchell Fwy Nth Bound',
       'Kwinana Fwy NB between Leach Hwy East Bound On - H015 & Cranford Av On - H015 Nth Bo

In [95]:
dataset['ID'] = dataset['Link']
dataset.head()

,Link,Length,DateTime,Volume,Speed,Occupancy,ID
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,16.0,80.000000,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,15.0,93.999998,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,15.0,93.999998,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,15.0,95.000000,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,14.0,96.999999,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...


In [96]:
dataset = dataset.replace({'ID':{
    'Kwinana Fwy NB between Kwinana Fwy Nth Bnd H015 Nth Bnd - H018 East Bnd & Kwinana Fwy Nth Bnd H018 W': 1,
    'Kwinana Fwy NB between Kwinana Fwy Nth Bnd H018 West Bnd - H015 Nth Bnd & Farrington Rd On - H015 Nt': 2,
    'Kwinana Fwy NB between Farrington Rd On - H015 Nth Bo & H015 Nth Bound - South St Off': 3,
    'Kwinana Fwy NB between H015 Nth Bound - South St Off & South St On - H015 Nth Bound': 4,
    'Kwinana Fwy NB between South St On - H015 Nth Bound & H015 Nth Bound - Leach Hwy Off': 5,
    'Kwinana Fwy NB between H015 Nth Bound - Leach Hwy Off & Leach Hwy West Bound On - H015': 6,
    'Kwinana Fwy NB between Leach Hwy West Bound On - H015 & Leach Hwy East Bound On - H015': 7,
    'Kwinana Fwy NB between Leach Hwy East Bound On - H015 & Cranford Av On - H015 Nth Bou': 8,
    'Kwinana Fwy NB between Cranford Av On - H015 Nth Bou & H015 Sth Bound - H548': 9,
    'Kwinana Fwy NB between H015 Sth Bound - H548 & Manning Rd - H547 On Kwinana Fwy Nth Bound': 10,
    'Kwinana Fwy NWB between Manning Rd - H547 On Kwinana Fwy Nth Bound & Canning Hwy - H549 On Kwinana F': 11,
    'Kwinana Fwy NB between Kwinana Fwy (northbound) Bus Ln From Canning Hwy: H013 On To H015 Northbound': 12,
    'Kwinana Fwy NB between Mill Pt Rd - H500 On Kwinana Fwy Nth Bound & Kwinana Fwy Nth Bound H503 Off -': 13,
    'Kwinana Fwy NB between Kwinana Fwy Nth Bound H503 Off - Mill Pt Rd & Mitchell Fwy Nth Bound': 14,
}})

In [98]:
dataset = dataset[['ID', 'Link', 'Length', 'DateTime',
                   'Volume', 'Speed', 'Occupancy']]

In [99]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6007680 entries, 0 to 6007679
Data columns (total 7 columns):
 #   Column     Dtype         
---  ------     -----         
 0   ID         int64         
 1   Link       object        
 2   Length     float64       
 3   DateTime   datetime64[ns]
 4   Volume     float64       
 5   Speed      float64       
 6   Occupancy  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 366.7+ MB


### Resample and interpolate time series data

In [89]:
# Change 0s in Volume column to NaNs
dataset['Volume'].replace(0, np.nan, inplace=True)

In [91]:
date_ls = pd.date_range(start='2018-01-01 00:00:00', 
                        end='2018-10-25 23:59:00', freq='1min')
date_ls2 = pd.to_datetime(dataset.DateTime)
date_ls.difference(date_ls2) # return missing dates

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

The data between 24 to 27 Mar is missing, we are going to replace it with the data between 17 to 20 Mar.

| Sat | Sun | Mon | Tue |
|-----|-----|-----|-----|
| 17  | 18  |  19 |  20 |
| 24  | 25  | 26  | 27  |

In [104]:
dat = dataset.set_index(['ID', 'DateTime'])
dat = dat.sort_index(); dat

Link  \
ID DateTime                                                                 
1  2018-01-01 00:00:00  Kwinana Fwy NB between Kwinana Fwy Nth Bnd H01...   
   2018-01-01 00:01:00  Kwinana Fwy NB between Kwinana Fwy Nth Bnd H01...   
   2018-01-01 00:02:00  Kwinana Fwy NB between Kwinana Fwy Nth Bnd H01...   
   2018-01-01 00:03:00  Kwinana Fwy NB between Kwinana Fwy Nth Bnd H01...   
   2018-01-01 00:04:00  Kwinana Fwy NB between Kwinana Fwy Nth Bnd H01...   
...                                                                   ...   
14 2018-10-25 23:55:00  Kwinana Fwy NB between Kwinana Fwy Nth Bound H...   
   2018-10-25 23:56:00  Kwinana Fwy NB between Kwinana Fwy Nth Bound H...   
   2018-10-25 23:57:00  Kwinana Fwy NB between Kwinana Fwy Nth Bound H...   
   2018-10-25 23:58:00  Kwinana Fwy NB between Kwinana Fwy Nth Bound H...   
   2018-10-25 23:59:00  Kwinana Fwy NB between Kwinana Fwy Nth Bound H...   

                        Length  Volume      Speed  Occupancy  
ID DateTime                                                   
1  2018-01-01 00:00:00   960.0     7.0  96.000000        1.0  
   2018-01-01 00:01:00   960.0     6.0  94.999998        1.0  
   2018-01-01 00:02:00   960.0     5.0  90.999999        0.8  
   2018-01-01 00:03:00   960.0     5.0  94.999997        0.8  
   2018-01-01 00:04:00   960.0     5.0  92.999999        1.0  
...                        ...     ...        ...        ...  
14 2018-10-25 23:55:00   567.0     6.0  82.999997        0.0  
   2018-10-25 23:56:00   567.0     6.0  78.000003        0.0  
   2018-10-25 23:57:00   567.0     5.0  78.000001        0.0  
   2018-10-25 23:58:00   567.0     6.0  74.000000        0.0  
   2018-10-25 23:59:00   567.0     7.0  82.999997        0.0  

[6007680 rows x 5 columns]

In [ ]:
test = dataset.groupby(['ID', 'DateTime']).transform(lambda v: v.ffill());test

,Link,Length,Volume,Speed,Occupancy
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,16.0,80.000000,1.8
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,15.0,93.999998,1.8
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,15.0,93.999998,1.8
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,15.0,95.000000,1.8
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,14.0,96.999999,1.8
...,...,...,...,...,...
6007675,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,5.0,96.999997,0.0
6007676,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,5.0,96.999997,0.0
6007677,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,5.0,94.000001,0.0
6007678,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,5.0,94.000001,0.0


In [ ]:
test[(test.ID==1) & (test.DateTime.dt.date)]

In [ ]:
# interpolate if possible
# for days with no data, use data from the same day last week